## Estimate the amount of time that ATBs are full 

In [1]:
import pandas 
import numpy 
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import datetime

# import functions for querying DOE and monte-carlo dataframes
from monte_carlo_utils import get_DOE_df, get_DOE_atb


In [2]:
# Dept. of Ecology data files
DOE_dir  = Path('/Users/rmueller/Data/MIDOSS/DeptOfEcology/')
DOE_2018_xlsx = DOE_dir/'MuellerTrans4-30-20.xlsx'

# ATB spreadsheet
atb_dir = Path('/Users/rmueller/Projects/MIDOSS/AIS/origin_destination_analysis')
atb_xlsx = atb_dir/'ATB_MMSIs.xlsx'

# Facility names and lat/lon information file
facilities_xlsx = Path(
    '/Users/rmueller/Data/MIDOSS/marine_transport_data/'
    'Oil_Transfer_Facilities.xlsx'
)

# Load Oil Attribution file 
oil_attribution_file = '/Users/rmueller/Data/MIDOSS/marine_transport_data/oil_attribution.yaml'

with open(oil_attribution_file) as file:
        oil_attrs = yaml.load(file, Loader=yaml.Loader)

In [3]:
bbl2gal = 42

In [4]:
atb_df={}
atb_df['MMSI'] = pandas.read_excel(
    atb_xlsx,
    sheet_name='DOE ATBs', 
    usecols="A,B"
    )
atb_df['Capacity'] = pandas.read_excel(
    atb_xlsx,
    sheet_name=0,
    nrows=21,
    usecols="A,F,I,J"
)

In [5]:
atb_df['Capacity']

,MMSI,Vessel length,Fuel Capacity,Cargo Capacity
0,303668000.0,NaN,NaN,"178,000 bbl"
1,316001221.0,NaN,NaN,NaN
2,316001223.0,41.45 m,NaN,NaN
3,316006101.0,NaN,NaN,NaN
4,316038089.0,NaN,NaN,NaN
5,338134000.0,136 ft,"150,000 gallons","185,000 barrels"
6,338616000.0,125.8 ft,"144,000 gallons","150,000 barrels"
7,366341000.0,NaN,NaN,"178,000 bbl"
8,366866930.0,NaN,NaN,NaN
9,366904000.0,NaN,NaN,"178,000 bbl"


In [6]:
atb_df['MMSI']['MMSI'][2,14,16,17]=numpy.NaN

In [7]:
atb_df['MMSI'] = atb_df['MMSI'].drop(index=13)
atb_df['MMSI'] = pandas.merge(
    left = atb_df['MMSI'],
    right = atb_df['Capacity'],
    how='left',
    on = 'MMSI'
)
atb_df['MMSI']

,DELIVERER,MMSI,Vessel length,Fuel Capacity,Cargo Capacity
0,ATB BARGE 550-2,369580000,NaN,NaN,"155,000 bbls"
1,ATB BARGE 550-3,369703000,NaN,NaN,"155,000 bbls"
2,ITB ISLAND TRADER,NaN,NaN,NaN,NaN
3,ATB BARGE KIRBY 185-01,338134000,136 ft,"150,000 gallons","185,000 barrels"
4,ATB BARGE 650-7,366341000,NaN,NaN,"178,000 bbl"
5,ATB BARGE ONEDREAM,368006870,111.3 ft,"101,383 gallons",NaN
6,ATB BARGE 650-10,366904000,NaN,NaN,"178,000 bbl"
7,ATB BARGE DBL 185,338616000,125.8 ft,"144,000 gallons","150,000 barrels"
8,ATB BARGE ALL ABOARD FOR A CURE,368013040,111.3 ft,"101,383 gallons",NaN
9,ATB BARGE 650-2,303668000,NaN,NaN,"178,000 bbl"


In [8]:
atb_df['MMSI']['Cargo Capacity'][0,1] = 155000
atb_df['MMSI']['Cargo Capacity'][3] = 185000
atb_df['MMSI']['Cargo Capacity'][4,6,9] = 178000
atb_df['MMSI']['Cargo Capacity'][7] = 150000
atb_df['MMSI']['Cargo Capacity'][11] = 80000
atb_df['MMSI']['Cargo Capacity'] = bbl2gal * atb_df['MMSI']['Cargo Capacity']
atb_df['MMSI'] = atb_df['MMSI'].drop(columns=["Vessel length",'Fuel Capacity','MMSI'])

In [9]:
[imports, exports]=get_DOE_atb(DOE_2018_xlsx, facilities_xlsx, transfer_type = 'cargo', facilities='selected')

this code not yet tested with fac_xls as input
cargo
cargo


In [16]:
# imports
import_df = {}
aggregation_functions = {'AntID': 'first', 'Deliverer': 'first', 'TransferQtyInGallon': 'sum'}
imports_sm = imports[['AntID','Deliverer','TransferQtyInGallon']]
imports_sm_AndID = imports_sm.groupby(imports['AntID']).aggregate(aggregation_functions)
import_df =  imports_sm_AndID[['Deliverer','TransferQtyInGallon']].groupby(
    'Deliverer').count().rename(columns={'TransferQtyInGallon':'Count'})
import_df.head()
import_df = pandas.merge(
    left = import_df,
    right = imports_sm_AndID[['Deliverer','TransferQtyInGallon']].groupby('Deliverer').sum(),
    how='left',
    on = 'Deliverer'
)
import_df['AverageQty'] = import_df['TransferQtyInGallon']/import_df['Count']
len(import_df)

13

In [17]:
# exports
export_df = {}
aggregation_functions = {'AntID': 'first', 'Receiver': 'first', 'TransferQtyInGallon': 'sum'}
exports_sm = exports[['AntID','Receiver','TransferQtyInGallon']]
exports_sm_AndID = exports_sm.groupby(exports['AntID']).aggregate(aggregation_functions)
export_df =  exports_sm_AndID[['Receiver','TransferQtyInGallon']].groupby(
    'Receiver').count().rename(columns={'TransferQtyInGallon':'Count'})
export_df.head()
export_df = pandas.merge(
    left = export_df,
    right = exports_sm_AndID[['Receiver','TransferQtyInGallon']].groupby('Receiver').sum(),
    how='left',
    on = 'Receiver'
)
export_df['AverageQty'] = export_df['TransferQtyInGallon']/export_df['Count']
len(export_df)

17

In [18]:
atb_df['MMSI'].head()

,DELIVERER,Cargo Capacity
0,ATB BARGE 550-2,6510000
1,ATB BARGE 550-3,6510000
2,ITB ISLAND TRADER,NaN
3,ATB BARGE KIRBY 185-01,7770000
4,ATB BARGE 650-7,7476000


In [19]:
exports_df

,Count,TransferQtyInGallon,AverageQty,DELIVERER,Cargo Capacity
0,42,206270400.0,4.911200e+06,ATB BARGE 550-2,6510000
1,6,27510000.0,4.585000e+06,ATB BARGE 550-3,6510000
2,32,120372000.0,3.761625e+06,ATB BARGE 650-10,7476000
3,4,22050000.0,5.512500e+06,ATB BARGE 650-2,7476000
4,6,34860000.0,5.810000e+06,ATB BARGE 650-7,7476000
5,30,63378000.0,2.112600e+06,ATB BARGE ALL ABOARD FOR A CURE,NaN
6,43,324366000.0,7.543395e+06,ATB BARGE DBL 185,6300000
7,1,3108000.0,3.108000e+06,ATB BARGE DBL 78,NaN
8,1,7560000.0,7.560000e+06,NaN,NaN
9,6,19740000.0,3.290000e+06,ATB BARGE DR. ROBERT J. BEALL,3360000


In [21]:
export_df = pandas.merge(
    left = export_df,
    right = atb_df['MMSI'],
    how='left',
    left_on = 'Receiver',
    right_on = 'DELIVERER'
)
import_df = pandas.merge(
    left = import_df,
    right = atb_df['MMSI'],
    how='left',
    left_on = 'Deliverer',
    right_on = 'DELIVERER'
)

In [22]:
export_df['PercentFull']=export_df['AverageQty']/export_df['Cargo Capacity']
import_df['PercentFull']=import_df['AverageQty']/import_df['Cargo Capacity']

In [24]:
fill_out = numpy.nanmedian(export_df['PercentFull'].tolist())
print(f'Export fill percentage: {fill_out}')
print(f'Number of export transfers {sum(export_df["Count"])}')

print(f'Import fill percentage: {fill_out}')
print(f'Number of import transfers {sum(import_df["Count"])}')

print(f'full/empty ratio: {sum(import_df["Count"])/sum(export_df["Count"])}')

Export fill percentage: 0.7458840763561677
Number of export transfers 299
Import fill percentage: 0.7458840763561677
Number of import transfers 188
full/empty ratio: 0.6287625418060201
